# 1. Prefer Exceptions to returning None 

Functions that return None to indicate special meaning are error prone because none and other values (e.g. zero, the empty string) all evaluate to false in conditional expressions.
In the below example, result_2 is perfectly valid and returns zero, but the handling is not proper.

In [2]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None
   
result_1 = divide(5, 0)
if not result_1:
    print("result_1 is not ok due to invalid inputs")
    
result_2 = divide(0, 5)
if not result_2:
    print("result_2 is not ok due to invalid inputs")

result_1 is not ok due to invalid inputs
result_2 is not ok due to invalid inputs


It is clearer to raise exceptions to indicate special situations instead of returning None. Expect the calling code to handle exceptions properly when they are documented.

In [7]:
def divide(a, b):
    """
    params: a: numerator
    params: b: denominator
    returns: the result when b is non-zero and ValueError when b is zero.
    """
    try:
        return a / b
    except ZeroDivisionError:
        return ValueError('Invalid inputs')
    
x, y = 5, 2
try:
    result = divide(x, y)
except ValueError:
    print("Invalid Inputs")
else:
    print(f"Result is {result}")
    

x, y = 5, 0
try:
    result = divide(x, y)
except ValueError:
    print("Invalid Inputs")
else:
    print(f"Result is {result}")

Result is 2.5
Result is Invalid inputs
